In [1]:
# 신경말 모듈 구성하기
# 싱경망은 데이터에 대한 연산을 수행하는 layer/module로 구성
# torch.nn namespace는 신경망을 구성하는데 필요한 모든 구성 요소를 제공

In [2]:
# FashionMNIST 데이터셋의 이미지들을 분류하는 신경망을 구성해보자
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
# 학습을 위한 장치 얻기
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [7]:
# 클래스 정의하기
# 신경망 모델을 nn.Module의 하위클래스로 정의하고, __init__에서 신경망 계층들을 초기화한다.
# nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # 입력 이미지를 1차원으로 평탄화 
        self.flatten = nn.Flatten()
        # 선형 레이어와 ReLU 활성화 함수를 쌓아올린 신경망 스택을 정의
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )

    # 입력 x를 받아서 평탄화 -> 신경망 스택을 통해 logits(출력)을 계산하여 반환
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
# NeuralNetwork의 인스턴스(instance)를 생성하고, 이를 device로 이동한 뒤, 구조를 출력
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
# 모델을 사용하기 위해 입력 데이터를 전달
# 일부 백그라운드 연산들과 함께 모델의 forward를 실행 -> model.forward() 직접 호출 X

X = torch.rand(1, 28, 28, device=device)
logits = model(X) 
# 모델에 입력을 전달하여 호출하면 2차원 텐서를 반환

pred_prob=nn.Softmax(dim=1)(logits) 
# 2차원 텐서의 dim=0은 각 분류(class)에 대한 원시(raw) 예측값 10개가, dim=1에는 각 출력의 개별 값들이 해당
# 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻기
y_pred=pred_prob.argmax(1)
print(f"Predicted class : {y_pred}")

Predicted class : tensor([6], device='cuda:0')


In [10]:
# 모델 계층(Layer)
# 28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보자
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [12]:
# nn.Flatten 계층을 초기화하여 각 28 x 28 의 2D 이미지를 784 픽셀 값을
# 갖는 연속된 배열로 변환.(dim=0의 미니배치 차원은 유지)
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [13]:
# nn.Linear, 선형 계층은 weight(가중치)와 bias를 사용하여 입력에
# 선형 변환(linear trnasformation)을 적용하는 모듈이다.
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [14]:
# 비선형 활성화(activiation)은 모델의 입력과 출력 사이에 
# 복잡한 관계(mapping)을 만든다. 비선형 활성화는 선형 변환 후에 적용되어,
# 비선형성(nonlinearity)을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.

# 이 모델에서는 nn.ReLU를 선형 계층들 사이에 사용하지만, 모델을 만들 때에는 비선형성을 가진 
# 다른 활성화를 도입할 수도 있다. 
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1083,  0.2952,  0.4350,  0.2116,  0.0100, -0.2616, -0.1152, -0.0945,
         -0.7499,  0.0501, -0.0270, -0.1861,  0.5941, -0.0679,  0.5229, -0.2782,
         -0.3581, -0.3154, -0.4844, -0.0475],
        [ 0.2189,  0.6365,  0.4665,  0.0094, -0.2461,  0.2281, -0.0622,  0.3277,
         -0.8499,  0.0362, -0.1775, -0.1701,  0.2127, -0.0209,  0.3490, -0.1376,
         -0.1457, -0.1268, -0.3420, -0.1304],
        [ 0.1781,  0.0248,  0.2415,  0.2587, -0.2078,  0.0401, -0.1512,  0.0922,
         -0.5532,  0.1703,  0.0397, -0.0415,  0.2452, -0.0304,  0.2178,  0.0129,
         -0.1386, -0.2998, -0.5263,  0.1276]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1083, 0.2952, 0.4350, 0.2116, 0.0100, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0501, 0.0000, 0.0000, 0.5941, 0.0000, 0.5229, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.2189, 0.6365, 0.4665, 0.0094, 0.0000, 0.2281, 0.0000, 0.3277, 0.0000,
         0.0362, 0.0000, 0.0000, 0.2127, 0.0000, 0.34

In [16]:
# nn.Sequential 은 순서를 갖는 모듈의 컨테이너이다.
# 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달된다.
# 순차 컨테이너(sequential container)를 사용하여 아래의 seq_modules 와 같은 신경망을 빠르게 만들 수 있다.
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
seq_modules

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=20, bias=True)
  (2): ReLU()
  (3): Linear(in_features=20, out_features=10, bias=True)
)

In [17]:
# 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 logits을 반환.
# logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale)된다.
# dim 매개변수는 값의 합이 1이 되는 차원을 나타낸다.
softmax = nn.Softmax(dim=1)
pred_prob = softmax(logits)
pred_prob

tensor([[0.1210, 0.0859, 0.1137, 0.0943, 0.1259, 0.0793, 0.0847, 0.1053, 0.0855,
         0.1044],
        [0.1250, 0.0876, 0.1100, 0.0953, 0.1196, 0.0834, 0.0841, 0.1099, 0.0786,
         0.1065],
        [0.1147, 0.0858, 0.1071, 0.1031, 0.1294, 0.0939, 0.0824, 0.1182, 0.0721,
         0.0933]], grad_fn=<SoftmaxBackward0>)

In [19]:
# 모델 매개변수
# 신경망 내부의 많은 계층들은 매개변수화 된다.
# 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어진다.
# nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적되며, 
# 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 된다.

# 이 예제에서는 각 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력.
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:-1]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0105, -0.0121, -0.0301,  ...,  0.0100, -0.0350,  0.0103],
        [ 0.0130, -0.0002,  0.0248,  ...,  0.0078, -0.0310,  0.0100],
        [ 0.0113,  0.0237, -0.0146,  ...,  0.0302,  0.0175, -0.0055],
        ...,
        [ 0.0336, -0.0094, -0.0254,  ..., -0.0123, -0.0258, -0.0178],
        [-0.0055,  0.0085,  0.0343,  ..., -0.0152, -0.0341,  0.0298],
        [-0.0295, -0.0322,  0.0031,  ..., -0.0232, -0.0204, -0.0111]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0174,  0.0123, -0.00